In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from time import time

In [2]:
from prophet import Prophet
from prophet.diagnostics import cross_validation
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.templates.autogen.forecast_config import (
    ComputationParam,
    EvaluationPeriodParam,
    ForecastConfig,
    MetadataParam,
)

In [3]:
import logging
from logging import getLogger
logger = getLogger('cmdstanpy')

In [4]:
def make_series(n, base_date='2020-01-01'):
    np.random.seed(0)
    df = pd.DataFrame()
    index = np.arange(n)
    time_index = pd.to_datetime(base_date) + pd.to_timedelta([f'{n} days' for n in index])
    df['ds'] = time_index.date
    a_trend = 1
    delta = np.where(index < 0.4 * n, a_trend / n, 2.0 * a_trend / n)
    b_day = np.where(time_index.day == 25, 1.2, 1.0)
    b_dow = np.where(
        time_index.dayofweek == 0,
        1.3,
        np.where(time_index.dayofweek == 6, 1.1, 1.0)
    )
    df['y'] = (100 + np.cumsum(delta)) * b_day * b_dow + (1.0 + 0.01 * np.random.randn(n))
    return df

In [5]:
n = 1000
df = make_series(n)
print(df.columns.values)
print(df['ds'].min(), df['ds'].max())

['ds' 'y']
2020-01-01 2022-09-26


## Prophet fit

In [6]:
p = int(0.90 * n)
df_train = df.iloc[0:p]
df_test= df.iloc[p:n]
print(df_train['ds'].min(), df_train['ds'].max())
print(df_test['ds'].min(), df_test['ds'].max())

2020-01-01 2022-06-18
2022-06-19 2022-09-26


In [7]:
logger.setLevel(logging.INFO)
s = time()
model = Prophet()
model.fit(df_train)
t = time()
print(t - s)

11:37:15 - cmdstanpy - INFO - Chain [1] start processing
11:37:15 - cmdstanpy - INFO - Chain [1] done processing


0.7320771217346191


In [8]:
s = time()
df_pred = model.predict(df_test)
t = time()
print(t - s)

0.6161530017852783


In [9]:
mae = mean_absolute_error(df_test['y'], df_pred['yhat'])
mape = mean_absolute_percentage_error(df_test['y'], df_pred['yhat'])
print(f"MAE  = {mae}")
print(f"MAPE = {mape}")

MAE  = 1.3837261962100948
MAPE = 0.011266327716615319


### Greykite

https://linkedin.github.io/greykite/docs/0.3.0/html/pages/stepbystep/0400_configuration.html

CV数 + バックテスト + 予測 の回数だけ学習する。

In [10]:
metadata_param = MetadataParam(time_col='ds', value_col='y', freq='D')
computation_param = ComputationParam(n_jobs=-1)
evaluation_period_param = EvaluationPeriodParam(
    test_horizon=0,
    cv_horizon=30,
    cv_max_splits=5
)
config = ForecastConfig(
    model_template=ModelTemplateEnum.SILVERKITE.name,
    metadata_param=metadata_param,
    computation_param=computation_param,
    evaluation_period_param=evaluation_period_param,
    forecast_horizon=n-p
)

In [11]:
forecaster = Forecaster()
s = time()
result = forecaster.run_forecast_config(df=df_train, config=config)
t = time()
print(t - s)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.9/site-packages/greykite/framework/pipeline/pipeline.py:181: UserWarning: No data selected for test (test_horizon=0). It is important to check out of sample performance
  warnings.warn("No data selected for test (test_horizon=0). "


16.121201276779175


In [12]:
df_pred = result.forecast.df.tail(n-p)
print(df_pred['ds'].min(), df_pred['ds'].max())

2022-06-19 00:00:00 2022-09-26 00:00:00


In [13]:
mae = mean_absolute_error(df_test['y'], df_pred['forecast'])
mape = mean_absolute_percentage_error(df_test['y'], df_pred['forecast'])
print(f"MAE  = {mae}")
print(f"MAPE = {mape}")

MAE  = 1.8712432229484515
MAPE = 0.01582689727568757


In [14]:
s = time()
df_future = result.timeseries.make_future_dataframe(periods=n - p, include_history=False)
df_pred = result.model.predict(df_future)
t = time()
print(t - s)

/usr/local/lib/python3.9/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning: Input data has many null values. Missing 100.00% of one input.
  warnings.warn(f"Input data has many null values. Missing {self.null_frac.max():.2%} of one input.",


0.7079308032989502


## Prophet cross_validation

In [15]:
logger.setLevel(logging.WARNING)
s = time()
cv = cross_validation(
    model,
    horizon='30 days',
    period='30 days',
)
t = time()
print(t - s)

  0%|          | 0/17 [00:00<?, ?it/s]

11.384675979614258
